In [1]:
import spacy
import pandas as pd
import re

In [2]:
data = pd.read_csv('pledges - pledges.csv', delimiter='|')

In [73]:
bag_of_words = {}
for index, row in data.iterrows():
    row = row['vehicleproperty_description_short']
    row = row.lower()
    row = re.sub(r'\W', ' ', row)
    row = re.sub(r'\d', ' ', row)
    row = re.sub(r'\s+', ' ', row)
    words = row.split(' ')
    for word in words: 
        if word in bag_of_words and len(word) > 1:
            bag_of_words[word] = bag_of_words[word] + 1
        else: 
            bag_of_words[word] = 1
# print(bag_of_words)
pd_bag = pd.DataFrame.from_dict(bag_of_words, orient='index', columns=[ 'count'])
pd_bag_sort = pd_bag[pd_bag['count']>1].sort_values('count', ascending=False)

pd_bag_sort.to_csv('data/bag_of_words.csv')
pd_bag_sort.count()

count    3385
dtype: int64

In [99]:
# bag_of_words_clean = pd.read_csv('data/bag_of_words_clean.csv')
# bag_of_words_clean_unique = pd.DataFrame(set(bag_of_words_clean['words'].values))
# bag_of_words_clean_unique.to_csv('data/bag_of_words_clean_unique.csv',  index=False)
bag_of_words_clean_unique = pd.read_csv('data/bag_of_words_clean_unique.csv')
bag_of_words_uniq = {}
for index,word in bag_of_words_clean_unique.iterrows():
    word = word['word']
    if word in bag_of_words:
        bag_of_words_uniq[word] = bag_of_words[word]
bag_of_words_uniq['klas']

1

In [94]:
import wordsegment
wordsegment.load()

In [100]:
from collections import Counter

bag_of_words_clean_unique = pd.read_csv('data/bag_of_words_clean_unique.csv')
wordsegment.UNIGRAMS.clear()
wordsegment.UNIGRAMS.update(bag_of_words_uniq)

# def pairs(iterable):
#     iterator = iter(iterable)
#     values = [next(iterator)]
#     for value in iterator:
#         values.append(value)
#         yield ' '.join(values)
#         del values[0]

# wordsegment.BIGRAMS.clear()
# wordsegment.BIGRAMS.update(Counter(pairs(tokenize(text))))
from wordsegment import _segmenter

def identity(value):
    return value

_segmenter.clean = identity
wordsegment.segment('Экскаватор ЕК 14-21год выпкска 2010заводской')

['Экскаватор ЕК 14-21год в', 'ыпкска 2010заводской']

In [50]:
nlp = spacy.load("ru_core_news_sm")

config = { "overwrite_ents": True, "validate": True }
ruler = nlp.add_pipe("entity_ruler", before="ner", config=config).from_disk("./patterns.jsonl", )

ents = []
for article in data['vehicleproperty_description_short'].values:
    article = str(article)
    doc = nlp(article)
    list_brands = [ent.ent_id_ for ent in doc.ents if ent.label_ == 'BRAND']
    ents.append(None if len(list_brands) == 0 else list_brands[0])

df2 = data.assign(Brand_NER=ents)

In [51]:
df2['Brand_NER'].value_counts()

,id,notificationreferencenumber,vehicleproperty_vin,vehicleproperty_description_short,nBody,nChas,BrandModel,BrandModel2,BrandModel3,Brand,Brand2,hz,hz.1,hz.2,hz.3,hz.4,hz.5,hz.6,hz.7,Brand_NER
0,1,2014-000-000001-888,SJNFAAJ10U2648684,Автомобиль,NaN,NaN,NaN,NaN,NaN,NISSAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,21,2014-000-000004-443,Y7A64430060802279,а/м КРАЗ 6443-0000080-022006 г.в.ПТС 50 ТТ 044...,NaN,NaN,NaN,KRAZ OTHER,KRAZ OTHER,KRAZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,23,2014-000-000004-443,NaN,Полуприцеп тяжеловоз ЧМЗАП-99901991 г.в.ПТС 59...,NaN,NaN,NaN,NaN,CHMZAP OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,25,2014-000-000005-728,JMZBK12F681713003,марка MAZDA 32007 года выпуска№ кузова JMZBK12...,NaN,NaN,NaN,NaN,NaN,FORD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mazda
4,26,2014-000-000006-281,XTA217230E0258955,ВАЗ 217230,NaN,NaN,NaN,NaN,LADA OTHER,ATLETIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,36740,2014-000-008915-818,1B4GP25B92B513140,Модель автомобиля: DODGE CARAVANгод выпуска:2002,NaN,NaN,NaN,NaN,NaN,CHRYSLER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dodge
9996,36741,2014-000-008919-331,MMBJNKB407D169678,Модель автомобиля: MITSUBISHI L200 DI-Dгод вып...,NaN,NaN,NaN,NaN,NaN,MITSUBISHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mitsubishi
9997,36742,2014-000-008920-502,WBADD11090BN64855,Модель автомобиля: BMW 520Iгод выпуска:1998,NaN,NaN,NaN,NaN,NaN,BMW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bmw
9998,36743,2014-000-008921-558,AHTFZ29G809033435,Модель автомобиля: TOYOTA HILUXгод выпуска:2008,NaN,NaN,NaN,NaN,NaN,TOYOTA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,toyota


In [52]:
df2.to_csv('with_ner.csv', index=False)

lada           648
toyota         388
hyundai        335
kia            289
volkswagen     269
              ... 
kaessbohrer      1
dacia            1
fliegl           1
asia             1
peterbilt        1
Name: Brand_NER, Length: 80, dtype: int64

In [53]:
df2['Brand_NER'].isnull().value_counts()

In [55]:
df3 = df2[df2['Brand_NER'].isnull()][df2['vehicleproperty_description_short'] != 'Автомобиль'][1400:1450]
df3

True     5370
False    4630
Name: Brand_NER, dtype: int64

In [ ]:
df3.loc[6252]['vehicleproperty_description_short']

In [ ]:

patterns = [

{"label": "BRAND", "pattern": [{"LOWER": "маз"}, {"IS_PUNCT": True},{'IS_DIGIT': True}], "id": "maz"},
{"label": "BRAND", "pattern": [{"LOWER": {"REGEX": "^маз-[0-9]*$"}}], "id": "maz"},
{"label": "BRAND", "pattern": [{"LOWER": "john"}, {"LOWER": "deere"}], "id": "john deere"}
]
ruler.add_patterns(patterns)

doc = nlp('Модель автомобиля: CHEVROLETKLAN (J200/Chevrol')
print(doc.ents)
print([ent.ent_id_ for ent in doc.ents])
print([(token.lemma_, token.tag_) for token in doc])